In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
import optuna

In [2]:
path = "/Users/amrtamer/Documents/Internship"
# path = "/content/drive/MyDrive/Internship"

In [3]:
data = pd.read_csv(path + r'/data/Volve/Volve_cleaned_prepared.csv')
if "Unnamed: 0" in data.columns:
    data = data.drop(columns="Unnamed: 0")
data.head()

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,linear_regressor,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.101891,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.101891,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.101891,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.00519,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.113263,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.00519,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,0.082298,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.00519,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."


In [4]:
well_info = dict(set([(j, i) for i, j in data[['WELL_BORE_CODE', 'well_name']].values.tolist()])) # creating a dictionary of well identiification with {code: name} structure
well_info

{7078: 'NO 15/9-F-11 H',
 5769: 'NO 15/9-F-5 AH',
 7405: 'NO 15/9-F-1 C',
 7289: 'NO 15/9-F-15 D',
 5351: 'NO 15/9-F-14 H',
 5599: 'NO 15/9-F-12 H'}

In [5]:
data.columns

Index(['well_name', 'WELL_BORE_CODE', 'date', 'prod_hrs', 'bhp', 'bht',
       'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht',
       'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'oil_rate',
       'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long',
       'Rolling_short', 'Rolling_medium', 'Rolling_long', 'Lag_short',
       'Lag_medium', 'Lag_long'],
      dtype='object')

In [6]:
identification_column = ['well_name', 'WELL_BORE_CODE', 'date']
feature_columns = ['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'Lag_short', 'Lag_medium', 'Lag_long', 'oil_rate']
target_column = ['oil_rate']

In [7]:
data = data[identification_column + feature_columns]
data.shape

(7987, 26)

In [8]:
data

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,Lag_short,Lag_medium,Lag_long,oil_rate
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.005190,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.102310
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.005190,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.102310
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.005190,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10...",0.177706
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.005190,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097...",-0.042238
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.005190,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615...",0.028674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,5769,NO 15/9-F-5 AH,2016-08-22,0.268405,-0.880933,-0.806036,-0.814175,1.149266,1.150153,-1.140330,...,-0.692802,-0.702316,-0.715245,-0.692202,-0.703021,-0.724022,"[-0.6852415797773639, -0.6943429268335604, -0....","[-0.6933422169244923, -0.687878486910018, -0.6...","[-0.7133929373653101, -0.7128670168291575, -0....",-0.683109
7983,5769,NO 15/9-F-5 AH,2016-08-23,0.268405,-0.880933,-0.806036,-0.814175,1.158798,1.150153,-1.136215,...,-0.689514,-0.699464,-0.713067,-0.690182,-0.699482,-0.722275,"[-0.6943429268335604, -0.6840874763785847, -0....","[-0.687878486910018, -0.6852415797773639, -0.6...","[-0.7128670168291575, -0.704496114962062, -0.6...",-0.677835
7984,5769,NO 15/9-F-5 AH,2016-08-24,0.268405,-0.880933,-0.806036,-0.814175,1.141085,1.150153,-1.137527,...,-0.687737,-0.697357,-0.711216,-0.688940,-0.696914,-0.720063,"[-0.6840874763785847, -0.6831086798251896, -0....","[-0.6852415797773639, -0.6943429268335604, -0....","[-0.704496114962062, -0.6974619277910209, -0.6...",-0.679069
7985,5769,NO 15/9-F-5 AH,2016-08-25,0.268405,-1.334973,-1.285316,-1.327391,1.140689,1.150153,-1.140776,...,-0.686410,-0.695555,-0.709500,-0.685783,-0.695013,-0.718130,"[-0.6831086798251896, -0.6778348655598816, -0....","[-0.6943429268335604, -0.6840874763785847, -0....","[-0.6974619277910209, -0.6908660077334403, -0....",-0.678646


In [9]:
data.isna().any()

well_name                False
WELL_BORE_CODE           False
date                     False
prod_hrs                 False
bhp                      False
bht                      False
dp_tubing                False
AVG_ANNULUS_PRESS        False
AVG_CHOKE_SIZE_P         False
whp                      False
wht                      False
choke_size_percentage    False
oil_vol                  False
gas_vol                  False
water_vol                False
linear_regressor         False
EMA_short                False
EMA_medium               False
EMA_long                 False
Rolling_short            False
Rolling_medium           False
Rolling_long             False
Lag_short                False
Lag_medium               False
Lag_long                 False
oil_rate                 False
dtype: bool